This is the workflow for uploading and cropping photos for the two cameras, Camera 5 and Nikon COOLPIX AW120.

You can upload new photos to the Uncropped Folder within this BOP Photoplots parent folder. It shouldn't matter if you reupload any duplicates, you can just overwrite them at this step. 

In [1]:
import csv
from pathlib import Path
from fractions import Fraction
from PIL import Image, ImageOps

# -----------------------
# FOLDER SELECTION (your pattern)
# -----------------------
BATCH_DATE = 20251002 #Enter as YYYYMMDD


In [2]:
# USER INPUTS / GLOBALS
# -----------------------
PARENT_BASE = Path("/workspaces/BOP_OCTC/Python/BOP Photoplots")
PARENT_IN  = PARENT_BASE / "Uncropped"
PARENT_OUT = PARENT_BASE / "Cropped"

# Camera height above ground (meters)
H_M = 2.0

# If True, use the smaller of (px/m X, px/m Y) to avoid under-cropping 1 m
USE_CONSERVATIVE = False

# If True, read EXIF FocalLength (tag 37386) and override focal length per image if present
USE_EXIF_FOCAL = True

# Target ground side in meters
TARGET_GROUND_M = 1.0
# Camera configs
CAMERA_CONFIG = {
    "Camera 5": {  # Coolpix AW100
        "sensor_w_mm": 6.17,
        "sensor_h_mm": 4.55,
        "focal_mm": 5.0,
        "extensions": (".jpg", ".jpeg", ".tif", ".tiff", ".png"),
    },
    "Nikon COOLPIX AW120": {
        "sensor_w_mm": 6.17,
        "sensor_h_mm": 4.55,
        "focal_mm": 4.3,
        "extensions": (".jpg", ".jpeg", ".tif", ".tiff", ".png"),
    },
}

# -----------------------
# HELPERS
# -----------------------
def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def exif_focal_mm_or_default(pil_image, default_f_mm: float) -> float:
    try:
        exif = pil_image.getexif()
        focal_tag = 37386
        if exif and focal_tag in exif:
            val = exif.get(focal_tag)
            if isinstance(val, Fraction):
                return float(val)
            if isinstance(val, tuple) and len(val) == 2:
                num, den = val
                return float(num) / float(den) if den else default_f_mm
            return float(val)
    except Exception:
        pass
    return default_f_mm

def compute_px_per_meter(img_w, img_h, sensor_w_mm, sensor_h_mm, focal_mm, conservative=False):
    ground_w_m = H_M * (sensor_w_mm / focal_mm)
    ground_h_m = H_M * (sensor_h_mm / focal_mm)
    px_per_m_x = img_w / ground_w_m
    px_per_m_y = img_h / ground_h_m
    px_per_m = min(px_per_m_x, px_per_m_y) if conservative else 0.5 * (px_per_m_x + px_per_m_y)
    return px_per_m_x, px_per_m_y, px_per_m

def center_crop_square(im, side_px):
    w, h = im.size
    side_px = min(int(round(side_px)), w, h)
    cx, cy = w // 2, h // 2
    half = side_px // 2
    left, top = cx - half, cy - half
    right, bottom = left + side_px, top + side_px
    return im.crop((left, top, right, bottom)), side_px

def process_folder(folder_name: str, cfg: dict):
    in_dir  = PARENT_IN / folder_name
    out_dir = PARENT_OUT / folder_name
    ensure_dir(out_dir)

    sensor_w = cfg["sensor_w_mm"]
    sensor_h = cfg["sensor_h_mm"]
    focal_mm_default = cfg["focal_mm"]
    valid_ext = cfg["extensions"]

    log_rows = []
    processed = 0
    skipped_existing = 0

    for entry in sorted(in_dir.iterdir()):
        if not entry.is_file() or entry.suffix.lower() not in valid_ext:
            continue

        base = entry.stem
        ext  = entry.suffix.lower()
        out_name = f"{base}_c{ext}"
        out_path = out_dir / out_name

        if out_path.exists():
            skipped_existing += 1
            continue

        with Image.open(entry) as im_raw:
            im = ImageOps.exif_transpose(im_raw)
            w, h = im.size
            focal_used = exif_focal_mm_or_default(im_raw, focal_mm_default) if USE_EXIF_FOCAL else focal_mm_default
            px_per_m_x, px_per_m_y, px_per_m = compute_px_per_meter(w, h, sensor_w, sensor_h, focal_used, conservative=USE_CONSERVATIVE)
            crop_px = px_per_m * TARGET_GROUND_M
            cropped, side_used = center_crop_square(im, crop_px)

            save_kwargs = {}
            if ext in (".jpg", ".jpeg"):
                save_kwargs.update(dict(quality=95, subsampling=1, optimize=True))
            cropped.save(out_path, **save_kwargs)
            processed += 1

            log_rows.append({
                "folder": folder_name,
                "file": entry.name,
                "out_file": out_name,
                "img_w_px": w,
                "img_h_px": h,
                "px_per_m_x": round(px_per_m_x, 3),
                "px_per_m_y": round(px_per_m_y, 3),
                "px_per_m_used": round(px_per_m, 3),
                "crop_side_px_requested": round(crop_px, 2),
                "crop_side_px_used": side_used,
                "approx_ground_side_m": round(side_used / px_per_m, 4),
                "height_m": H_M,
                "focal_mm_default": focal_mm_default,
                "focal_mm_used": round(focal_used, 3),
                "sensor_w_mm": sensor_w,
                "sensor_h_mm": sensor_h,
            })

    if log_rows:
        log_name = f"crop_log_{BATCH_DATE}.csv"
        log_path = out_dir / log_name
        with open(log_path, "w", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=list(log_rows[0].keys()))
            writer.writeheader()
            writer.writerows(log_rows)

    print(f"[{folder_name}] processed: {processed}, skipped existing: {skipped_existing}, log: crop_log_{BATCH_DATE}.csv, out: {out_dir}")

# -----------------------
# MAIN
# -----------------------
if __name__ == "__main__":
    for folder, cfg in CAMERA_CONFIG.items():
        process_folder(folder, cfg)

[Camera 5] processed: 25, skipped existing: 0, log: crop_log_20251002.csv, out: /workspaces/BOP_OCTC/Python/BOP Photoplots/Cropped/Camera 5
[Nikon COOLPIX AW120] processed: 0, skipped existing: 135, log: crop_log_20251002.csv, out: /workspaces/BOP_OCTC/Python/BOP Photoplots/Cropped/Nikon COOLPIX AW120


After processing, run the following codes in the terminal:

1:
git add .

2:
git commit -m "Batching more photos"

3:
git push main